In [1]:
!pip install turicreate
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install sklearn

In [2]:
import turicreate as tc
import pandas as pd
from turicreate.toolkits.recommender.util import precision_recall_by_user

In [3]:
sf = tc.SFrame.read_csv('italyRatings.csv', verbose = False)
sf = sf.remove_column('rating_id') #remove the db id
sf2 = tc.SFrame.read_csv('italyTrails.csv', verbose = False)
sf2 = sf2.select_columns(['trail_id', 'name'])
sf = sf.join(sf2, on='trail_id', how='inner')
sf = sf.rename({'trail_id': 'item_id'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria Turicreate
sf = sf.rename({'name': 'item_name'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria Turicreate
sf2 = sf2.rename({'trail_id': 'item_id'})

#Splitting Dataset
train, test = tc.recommender.util.random_split_by_user(sf, user_id = "user_id",item_id = "item_id")
#train.head

## Item Based Collaborative Filtering - Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare l'item based recommender system sul dataset, inoltre proviamo a buildare il modello utilizzando dei parametri differenti per testare l'efficacia del sistema

In [4]:
item_based_cf_model = tc.recommender.item_similarity_recommender.create(train, user_id = 'user_id' , item_id = 'item_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'jaccard' ,
                                                          threshold = 0.001, verbose = False)
    
ibcf_similarity = item_based_cf_model.get_similar_items(k=3, verbose = False)
ibcf_similarity = ibcf_similarity.join(sf2, on='item_id', how='inner')
ibcf_similarity = ibcf_similarity.join(sf2, on={'similar':'item_id'}, how='inner')
ibcf_similarity = ibcf_similarity.rename({'name': 'item_name', 'name.1': 'similar_name'})
ibcf_similarity.print_rows(num_rows=20, num_columns=6, max_row_width = 150)
ibcf_recommend = item_based_cf_model.recommend(k=3, verbose = False)
ibcf_recommend = ibcf_recommend.join(sf2, on='item_id', how='inner')
ibcf_recommend.print_rows(num_rows=20, num_columns=5, max_row_width = 150)

#pr_result = precision_recall_by_user(test, ibcf_recommend)
#pr_result.print_rows(num_rows=500, num_columns=5, max_row_width = 150)



+---------+---------+----------------------+------+-------------------------------+-------------------------------+
| item_id | similar |        score         | rank |           item_name           |          similar_name         |
+---------+---------+----------------------+------+-------------------------------+-------------------------------+
|   917   |   1292  | 0.13015872240066528  |  1   | Sentiero Azzurro: Vernazza... | Sentiero Azzurro: Cornigli... |
|   917   |   1851  | 0.09446251392364502  |  2   | Sentiero Azzurro: Vernazza... |      Manarola - Corniglia     |
|   917   |   2550  |  0.0450819730758667  |  3   | Sentiero Azzurro: Vernazza... | Sentiero Azzurro: Vernazza... |
|   1292  |   917   | 0.13015872240066528  |  1   | Sentiero Azzurro: Cornigli... | Sentiero Azzurro: Vernazza... |
|   1292  |   1851  | 0.10819673538208008  |  2   | Sentiero Azzurro: Cornigli... |      Manarola - Corniglia     |
|   1292  |   2790  | 0.06696426868438721  |  3   | Sentiero Azzurro: Co

## User Based Collaborative Filtering - Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare l'user based recommender system sul dataset, utilizzeremo la stessa tipologia di modello utilizzata in precedenza ma con i parametri user_id e item_id invertiti come suggerisce la documentazione. Inoltre, come fatto nel caso precedente,  testiamo anche diversi parametri per valutare il sistema con l'efficacia migliore

In [5]:
user_based_cf_model = tc.recommender.item_similarity_recommender.create(train, user_id = 'item_id' , item_id = 'user_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'cosine' ,
                                                          threshold = 0.001, verbose = False)
    
ubcf_similarity = user_based_cf_model.get_similar_items(k=5, verbose = False)
ubcf_similarity.print_rows(num_rows=20, num_columns=4)
ubcf_recommendation = user_based_cf_model.recommend(k=5, verbose = False)
ubcf_recommendation = ubcf_recommendation.join(sf2, on='item_id', how='inner')
ubcf_recommendation = ubcf_recommendation.sort([('user_id', False), ('score', False)])
ubcf_recommendation = ubcf_recommendation.remove_column('rank')
ubcf_recommendation.print_rows(num_rows=20, num_columns=5, max_row_width = 150)


#pr_result = precision_recall_by_user(test, ubcf_recommendation, cutoffs=[3, 5])
#pr_result.print_rows(num_rows=500, num_columns=5, max_row_width = 150)


+---------+---------+-------+------+
| user_id | similar | score | rank |
+---------+---------+-------+------+
|   958   |   946   |  1.0  |  1   |
|   958   |   995   |  1.0  |  2   |
|   958   |   947   |  1.0  |  3   |
|   958   |   930   |  1.0  |  4   |
|   958   |   937   |  1.0  |  5   |
|   934   |   1095  |  1.0  |  1   |
|   934   |   979   |  1.0  |  2   |
|   934   |   942   |  1.0  |  3   |
|   934   |   1103  |  1.0  |  4   |
|   934   |   928   |  1.0  |  5   |
|   995   |   958   |  1.0  |  1   |
|   995   |   930   |  1.0  |  2   |
|   995   |   947   |  1.0  |  3   |
|   995   |   937   |  1.0  |  4   |
|   995   |   998   |  1.0  |  5   |
|   947   |   958   |  1.0  |  1   |
|   947   |   930   |  1.0  |  2   |
|   947   |   995   |  1.0  |  3   |
|   947   |   937   |  1.0  |  4   |
|   947   |   998   |  1.0  |  5   |
+---------+---------+-------+------+
[22257 rows x 4 columns]

+---------+---------+---------------------+-------------------------------+
| item_id 

## Matrix Factorization Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare la matrix factorization sul dataset, alternativa il cui approccio alla base risulta simile al Collaborative Filtering. Proviamo anche qui a far runnare il modello con diversi parametri

In [6]:
matrix_factorization_model = tc.ranking_factorization_recommender.create(train, target = 'user_rating' ,
                                                         ranking_regularization = 0.1 ,
                                                         unobserved_rating_value = 1,
                                                         max_iterations = 50, solver = "sgd", 
                                                         verbose = False)

matrix_similarity = matrix_factorization_model.get_similar_items(k = 5, verbose = False)
matrix_similarity = matrix_similarity.join(sf2, on='item_id', how='inner')
matrix_similarity = matrix_similarity.join(sf2, on={'similar':'item_id'}, how='inner')
matrix_similarity = matrix_similarity.rename({'name': 'item_name', 'name.1': 'similar_name'})
matrix_similarity.print_rows(num_rows=20, num_columns=6, max_row_width=150)
matrix_recommendation = matrix_factorization_model.recommend(k = 5, verbose = False)
matrix_recommendation = matrix_recommendation.join(sf2, on='item_id', how='inner')
matrix_recommendation.print_rows(num_rows=20, num_columns=5, max_row_width=150)

#pr_result = precision_recall_by_user(test, matrix_recommendation, cutoffs=[10])
#pr_result.print_rows(num_rows=500, num_columns=5, max_row_width = 150)

+---------+---------+--------------------+------+-------------------------------+--------------------------------+
| item_id | similar |       score        | rank |           item_name           |          similar_name          |
+---------+---------+--------------------+------+-------------------------------+--------------------------------+
|   917   |   4265  | 0.9771016240119934 |  1   | Sentiero Azzurro: Vernazza... | Bric Malanotte - Bricco Sa...  |
|   917   |  15810  | 0.9442564249038696 |  2   | Sentiero Azzurro: Vernazza... |         Monte Pasubio          |
|   917   |   4488  | 0.9196463227272034 |  3   | Sentiero Azzurro: Vernazza... | Costa d'Oneglia, Gorleri, ...  |
|   917   |  16422  | 0.7941082715988159 |  4   | Sentiero Azzurro: Vernazza... | Grezzana - Azzago - Monte ...  |
|   917   |   8763  | 0.7345039248466492 |  5   | Sentiero Azzurro: Vernazza... | San Lazzaro - Monte Molign...  |
|   1292  |  14360  | 0.9735556840896606 |  1   | Sentiero Azzurro: Cornigli... 

## Content Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare un Content Based Recommender sul dataset, in particolare lo faremo sul file trails.csv che contiene tutte le feature piu' importanti di un percorso (circa 60 in totale)

In [7]:
trails = tc.SFrame.read_csv('italyTrails.csv', verbose = False)
trails = trails.remove_columns({'image_link', 'name', 'length_km', 'elevationm'}) #remove the db id and other fields
trails = trails.rename({'trail_id': 'item_id'}) #rinomiano il trail_id in item_id in maniera tale da conformarci con la libreria 
train, test = tc.recommender.util.random_split_by_user(sf)
content_based_model = tc.recommender.item_content_recommender.create(item_data = trails, item_id="item_id", 
                                                                     observation_data = train, user_id="user_id", 
                                                                     target="user_rating", 
                                                                     verbose = False)
content_based_similiarity = content_based_model.get_similar_items(verbose = False)
content_based_similiarity = content_based_similiarity.join(sf2, on='item_id', how='inner')
content_based_similiarity = content_based_similiarity.join(sf2, on={'similar':'item_id'}, how='inner')
content_based_similiarity = content_based_similiarity.rename({'name': 'item_name', 'name.1': 'similar_name'})
content_based_similiarity.print_rows(num_rows=20, num_columns=6, max_row_width=150)
content_based_recommend = content_based_model.recommend(verbose = False)
content_based_recommend = content_based_recommend.join(sf2, on='item_id', how='inner')
content_based_recommend.print_rows(num_rows=20, num_columns=5, max_row_width=150)



+---------+---------+-------------------+------+-------------------------------+-------------------------------+
| item_id | similar |       score       | rank |           item_name           |          similar_name         |
+---------+---------+-------------------+------+-------------------------------+-------------------------------+
|   917   |   2672  | 0.957446813583374 |  1   | Sentiero Azzurro: Vernazza... |      Monterosso - Levanto     |
|   917   |   5304  | 0.936170220375061 |  2   | Sentiero Azzurro: Vernazza... |   Cala Domestica da Buggerru  |
|   917   |   3278  | 0.936170220375061 |  3   | Sentiero Azzurro: Vernazza... | Sentiero Azzurro: Riomaggi... |
|   917   |   2395  | 0.936170220375061 |  4   | Sentiero Azzurro: Vernazza... | Riomaggiore - Portovenere ... |
|   917   |   5274  | 0.936170220375061 |  5   | Sentiero Azzurro: Vernazza... | Capo Spartivento Coastal Path |
|   917   |   8555  | 0.936170220375061 |  6   | Sentiero Azzurro: Vernazza... | Sentiero degli 

## Popular Based Recommender System
##### In questo sottoparagrafo proviamo ad utilizzare il Recommender system piu' semplice: il popular based. Le raccomandazioni fornite sono utilii per superare il problema del Cold Start e per spingere l'utente a recensire piu' item possibili al fine di ottenere in futuro delle raccomandazioni che siano il piu' possibile pertinenti.

In [8]:
popular_based_model = tc.popularity_recommender.create(sf, target = 'user_rating', verbose = False)
popresult = popular_based_model.recommend()
popresult = popresult.join(sf2, on='item_id', how='inner')
popresult.print_rows(num_rows=20, num_columns=5)

recommendations finished on 1000/5385 queries. users per second: 455373

recommendations finished on 2000/5385 queries. users per second: 469153

recommendations finished on 3000/5385 queries. users per second: 472292

recommendations finished on 4000/5385 queries. users per second: 472925

recommendations finished on 5000/5385 queries. users per second: 431258

+---------+---------+-------+------+-------------------------------+
| user_id | item_id | score | rank |              name             |
+---------+---------+-------+------+-------------------------------+
|   958   |   4205  |  5.0  |  1   |      Vernazza - La Spezia     |
|   958   |   4196  |  5.0  |  2   | Sturla - Forte Richelieu -... |
|   958   |   4101  |  5.0  |  3   | Roc de l'Ormea - Monte Grosso |
|   958   |   3785  |  5.0  |  4   | La Mezza delle due Perle -... |
|   958   |   3782  |  5.0  |  5   | Traversata delle Alpi: Rif... |
|   958   |   3772  |  5.0  |  6   | Colle di Cervo via Sentier... |
|   958   |   3712  |  5.0  |  7   |      Alta Via Ventimiglia     |
|   958   |   3666  |  5.0  |  8   |  Colle del Prione-Nasino (SV) |
|   958   |   3548  |  5.0  |  9   | Passo del Lupo, Monte Gott... |
|   958   |   3325  |  5.0  |  10  |       Anello Monte Penna      |
|   934   |   4205  |  5.0  |  1   |      Vernazza - La Spezia     |
|   934   |   4196  |  5.0  |  2  

## Valutazione Sperimentale
##### In questo sottoparagrafo analizziamo la valutazione sperimentale dei Recommender System, si puo' notare come la Precision sia molto bassa per tutti i modelli (ma dai dati limitati sicuramente non abbondanti e' normale) mentre la Recall e' sicuramente molto piu' accettabile in relazione ai dati specialmente nella Matrix Factorization e nell'Item Based. L'item based e' il sistema che performa meglio nel complesso (era da aspettarselo per le ragioni discusse a lezione)

In [9]:
def binary_ratings(rating):
    if (rating<=3):
        return 0
    elif (rating>=4):
        return 1

In [10]:
pr = tc.SFrame.read_csv('italyRatings.csv', verbose = False)
pr.head()
ratings = pr.apply(lambda x: binary_ratings(x['user_rating'])) #trasformo i rating in binary
pr = pr.add_column(ratings, 'ratings')
pr.head()
pr = pr.remove_column('user_rating') #remove the db id
pr = pr.rename({'ratings': 'user_rating'})
pr = pr.rename({'trail_id': 'item_id'})



pr_train, pr_test = tc.recommender.util.random_split_by_user(pr, user_id = "user_id",item_id = "item_id")

pr_train.head()

rating_id,item_id,user_id,user_rating
1105,917,958,1
1106,917,934,1
1107,917,995,0
1108,917,947,0
1109,917,1003,1
1110,917,1087,1
1111,917,1088,1
1112,917,935,1
1113,917,1085,1
1114,917,949,1


In [11]:
item_based_cf_model = tc.recommender.item_similarity_recommender.create(pr_train, user_id = 'user_id' , item_id = 'item_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'jaccard' ,
                                                          #threshold = 0.001
                                                            verbose = False)

In [12]:
content_based_model = tc.recommender.item_content_recommender.create(item_data = trails, item_id="item_id", 
                                                                     observation_data = pr_train, user_id="user_id", 
                                                                     target="user_rating", 
                                                                     verbose = False)

In [13]:
matrix_factorization_model = tc.ranking_factorization_recommender.create(pr_train, target = 'user_rating' ,
                                                         #ranking_regularization = 0.1 ,
                                                         #unobserved_rating_value = 1,
                                                         max_iterations = 100, solver = "sgd", 
                                                         verbose = False)

In [14]:
user_based_cf_model = tc.recommender.item_similarity_recommender.create(pr_train, user_id = 'item_id' , item_id = 'user_id' , 
                                                          target = 'user_rating' , user_data = None ,
                                                          item_data = None , nearest_items = None , similarity_type = 'cosine' ,
                                                          #threshold = 0.001
                                                            verbose = False)

In [15]:
tc.recommender.util.compare_models(pr_test, [content_based_model, matrix_factorization_model, user_based_cf_model, item_based_cf_model], ["content_based_model", "matrix_factorization_model", "user_based_cf_model", "item_based_cf_model"], user_sample = 1 , metric='precision_recall', verbose = False)





[{'precision_recall_by_user': Columns:
  	user_id	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 4824
  
  Data:
  +---------+--------+-----------+--------+-------+
  | user_id | cutoff | precision | recall | count |
  +---------+--------+-----------+--------+-------+
  |   923   |   1    |    0.0    |  0.0   |   1   |
  |   923   |   2    |    0.0    |  0.0   |   1   |
  |   923   |   3    |    0.0    |  0.0   |   1   |
  |   923   |   4    |    0.0    |  0.0   |   1   |
  |   923   |   5    |    0.0    |  0.0   |   1   |
  |   923   |   6    |    0.0    |  0.0   |   1   |
  |   923   |   7    |    0.0    |  0.0   |   1   |
  |   923   |   8    |    0.0    |  0.0   |   1   |
  |   923   |   9    |    0.0    |  0.0   |   1   |
  |   923   |   10   |    0.0    |  0.0   |   1   |
  +---------+--------+-----------+--------+-------+
  [4824 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to 